In [21]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.inference import ApproxInference

In [22]:
# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianNetwork([('LB', 'D'), ('UR', 'D'), ('D', 'OR')])

# Defining individual CPDs.
cpd_LB = TabularCPD(variable='LB', variable_card=2, values=[[0.05], [0.95]])
cpd_UR = TabularCPD(variable='UR', variable_card=2, values=[[0.4], [0.6]])


cpd_D = TabularCPD(variable='D', variable_card=2,
                   values=[[0.9, 0.5, 0.3, 0.01],
                           [0.1, 0.5, 0.7, 0.99]],
                  evidence=['LB', 'UR'],
                  evidence_card=[2, 2])

cpd_OR = TabularCPD(variable='OR', variable_card=2,
                   values=[[0.9, 0.1],
                           [0.1, 0.9]],
                   evidence=['D'],
                   evidence_card=[2])

# Associating the CPDs with the network
model.add_cpds(cpd_LB, cpd_UR, cpd_D, cpd_OR)

# check_model checks for the network structure and CPDs and verifies that the CPDs are correctly
# defined and sum to 1.
model.check_model()
model.get_independencies()

(UR ⟂ LB)
(UR ⟂ OR | D)
(UR ⟂ OR | LB, D)
(LB ⟂ UR)
(LB ⟂ OR | D)
(LB ⟂ OR | UR, D)
(OR ⟂ UR, LB | D)
(OR ⟂ LB | UR, D)
(OR ⟂ UR | LB, D)

In [23]:
infer = VariableElimination(model)
g_dist = infer.query(['D'])
print(infer.query(['LB'], evidence={'OR': 0}))

+-------+-----------+
| LB    |   phi(LB) |
+=======+===========+
| LB(0) |    0.1413 |
+-------+-----------+
| LB(1) |    0.8587 |
+-------+-----------+


In [29]:
infer = ApproxInference(model)
print(infer.query('LB',n_samples=10000, evidence={'OR': 0}))

100%|██████████| 10000/10000 [00:00<00:00, 65145.33it/s]

+-------+-----------+
| LB    |   phi(LB) |
+=======+===========+
| LB(0) |    0.1427 |
+-------+-----------+
| LB(1) |    0.8573 |
+-------+-----------+


In [25]:
model.get_cpds()

[<TabularCPD representing P(LB:2) at 0x12f2204d0>,
 <TabularCPD representing P(UR:2) at 0x12f221090>,
 <TabularCPD representing P(D:2 | LB:2, UR:2) at 0x12f221110>,
 <TabularCPD representing P(OR:2 | D:2) at 0x12f2211d0>]